In [ ]:
import os
import sys
import copy
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
sys.path.append(os.path.join(os.path.pardir, 'gotmtool'))
sys.path.append(os.path.join(os.path.pardir, 'lesview'))
from gotmtool import *
from lesview import *
from sbl_bbl import *

In [ ]:
casename = 'lsc_ymc22_sbl_bbl_v2'
turbmethods = ['SMCLT-H15', 'SMCLT-KC04', 'KPPLT-LF17', 'KPPLT-LF17-E']

datapath = os.path.join(os.path.pardir, 'oceananigans', '{:s}'.format(casename))
filepath = os.path.join(datapath, 'averages.jld2')
data_pfl = OceananigansDataProfile(filepath=filepath)
ds_ocgn  = data_pfl.dataset

ds_gotm = {}
gotm_dir   = os.path.join(os.path.pardir, 'gotm', 'run', '{:s}'.format(casename))
for turbmethod in turbmethods:
    gotm_sim  = Simulation(path=os.path.join(gotm_dir, '{:s}-Rlx'.format(turbmethod)))
    ds_gotm[turbmethod] = gotm_sim.load_data()

In [ ]:
figpath  = 'overview_{:s}'.format(casename)
os.makedirs(figpath, exist_ok=True)

In [ ]:
g = 9.81
H = 30
N2 = 1.962e-4
bstar = N2 * H
u10 = 8
cd = 1.25e-3
rhoa = 1.225
rhoo = 1026
alphaT = 2.0e-4
lat = 45
tau = rhoa/rhoo*cd*u10*u10
ustar = np.sqrt(tau)
Ti = inertial_period(lat)
print('Ti = {:6.3f}'.format(Ti))

In [ ]:
print('---- Oceananigans ----')
N2_ocgn = ds_ocgn.data_vars['b'].differentiate(coord='z')/N2
tslice1, tslice2 = get_tslice(N2_ocgn, Ti)
# print('---- GOTM ----')
# tslice1_gotm = {}
# tslice2_gotm = {}
# for turbmethod in turbmethods:
#     print('-- {:s}'.format(turbmethod))
#     N2_gotm = ds_gotm[turbmethod].data_vars['temp'][:,:,0,0].differentiate(coord='z')*alphaT*g/N2
#     tslice1_gotm[turbmethod], tslice2_gotm[turbmethod] = get_tslice(N2_gotm, Ti)

In [ ]:
labels = dict(
    wb = '$10^3\overline{w^\prime b^\prime}/u_*b_*$',
    wu = '$\overline{w^\prime u^\prime}/u_*^2$',
    wv = '$\overline{w^\prime v^\prime}/u_*^2$',
)
das_ocgn = dict(
    wb = (ds_ocgn.data_vars['wb']+ds_ocgn.data_vars['wbsb'])/ustar/bstar*10**3,
    wu = (ds_ocgn.data_vars['wu']+ds_ocgn.data_vars['wusb'])/ustar**2,
    wv = (ds_ocgn.data_vars['wv']+ds_ocgn.data_vars['wvsb'])/ustar**2,
    )
das_gotm = {}
for turbmethod in turbmethods:
    ds = ds_gotm[turbmethod]
    das_gotm[turbmethod] = dict(
        wb = get_flux(ds.data_vars['temp'].squeeze(),
                      ds.data_vars['nuh'].squeeze(),
                      ds.data_vars['gamh'].squeeze())*alphaT*g/ustar/bstar*10**3,
        wu = (get_flux(ds.data_vars['u'].squeeze(),
                      ds.data_vars['num'].squeeze(),
                      ds.data_vars['gamu'].squeeze())
              -ds.data_vars['nucl'].squeeze()*ds.data_vars['dusdz'].squeeze())/ustar**2,
        wv = (get_flux(ds.data_vars['v'].squeeze(),
                      ds.data_vars['num'].squeeze(),
                      ds.data_vars['gamv'].squeeze())
              -ds.data_vars['nucl'].squeeze()*ds.data_vars['dvsdz'].squeeze())/ustar**2,
    )

In [ ]:
colors = ['tab:blue', 'tab:red', 'tab:purple', 'tab:purple']
linestyles = ['-', '-', '-', '--']
kw_line_les  = dict(linestyle='-', color='k')
kw_line_gotm = {}
for i, turbmethod in enumerate(turbmethods):
    kw_line_gotm[turbmethod] = dict(linestyle=linestyles[i], color=colors[i])
ylabels = dict(
    wb = 'Depth [m]',
    wu = '',
    wv = '',
)
abc = 'abcdef'
if 'rf' in casename:
    loc = 'lrllrl'
else:
    loc = 'llllll'
loc_ha = {'l': 'left', 'r': 'right'}
loc_x = {'l': 0.1, 'r': 0.9}

fig, axarr = plt.subplots(2, 3, sharex='col', sharey='row')
fig.set_size_inches([7, 5])

for i, var in enumerate(das_ocgn.keys()):
    ax = axarr[0, i]
    da = das_ocgn[var].sel(time=tslice1).mean(dim='time')
    da.plot(ax=ax, y=da.dims[0], **kw_line_les)
    for turbmethod in turbmethods:
        da = das_gotm[turbmethod][var].sel(time=tslice1).mean(dim='time')
        da.plot(ax=ax, y=da.dims[0], **kw_line_gotm[turbmethod])
    ax.set_title('')
    ax.set_ylabel(ylabels[var])
    ax.set_ylim([-H, 0])
    
    ax = axarr[1, i]
    da = das_ocgn[var].sel(time=tslice2).mean(dim='time')
    da.plot(ax=ax, y=da.dims[0], **kw_line_les, label='LES')
    for turbmethod in turbmethods:
        da = das_gotm[turbmethod][var].sel(time=tslice2).mean(dim='time')
        da.plot(ax=ax, y=da.dims[0], **kw_line_gotm[turbmethod], label=turbmethod[6:])
    ax.set_title('')
    ax.set_ylabel(ylabels[var])
    ax.set_ylim([-H, 0])
    ax.set_xlabel(labels[var])
    if i == 0:
        ax.legend(loc='lower left', fontsize=9)
for i in np.arange(2):
    for j in np.arange(3):
        k = i*3+j
        ax = axarr[i,j]
        ax.text(loc_x[loc[k]], 0.9, '({:s})'.format(abc[k]), transform=ax.transAxes, va='top', ha=loc_ha[loc[k]])

plt.tight_layout()
figname = os.path.join(figpath, 'profiles_relax')
fig.savefig(figname, dpi = 300, facecolor='w')